In [2]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

warnings.filterwarnings('ignore')

### Load Data

In [4]:
causeway = pd.read_excel('./output/causeway.xlsx')

df_customer_dim_with_affinity_score = pd.read_feather('./input/customer_dim_with_affinity_score.feather').rename(columns={'FCID':'Stock_Point_ID'})
df_customer_dim_with_affinity_score['Stock_Point_ID'] = df_customer_dim_with_affinity_score['Stock_Point_ID'].astype(int)

col1 = ['Latitude', 'Longitude','TotalSKUs', 'AvgSKUScore', 'TotalEstimatedVolume', 'RFcount', 'HighValueSKUs', 
        'HighValueAvgScore', 'HighValueTotalScore', 'HighValueEstimatedVolume', 'ExpressSKUs', 
        'CoreSKUs', 'CustomerAffinityScore_Raw', 'CustomerAffinityScore_Standardized', 'CustomerAffinityRank']

for col in col1: 
    df_customer_dim_with_affinity_score[col] = pd.to_numeric(df_customer_dim_with_affinity_score[col], errors='coerce').fillna(0)

   
# Replace invalid latitude values with NaN
df_customer_dim_with_affinity_score.loc[
    (df_customer_dim_with_affinity_score['Latitude'] < -90) |
    (df_customer_dim_with_affinity_score['Latitude'] > 90),
    'Latitude'
] = 0.0

df_customer_dim_with_affinity_score.loc[
    (df_customer_dim_with_affinity_score['Longitude'] < -180) |
    (df_customer_dim_with_affinity_score['Longitude'] > 180),
    'Longitude'
] = 0.0   

customers_df = df_customer_dim_with_affinity_score.merge(causeway[['CustomerID']].drop_duplicates(), how='inner', on='CustomerID')

### Main

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore')

# Generate sample customer data
def generate_sample_data(n_customers=100, city_center=(40.7589, -73.9851), spread_km=20):
    """Generate sample customer data around a city center."""
    np.random.seed(42)
    
    # Generate random coordinates around city center
    # Convert km to approximate degrees (rough approximation)
    lat_spread = spread_km / 111.0  # 1 degree lat ≈ 111 km
    lon_spread = spread_km / (111.0 * np.cos(np.radians(city_center[0])))
    
    latitudes = np.random.normal(city_center[0], lat_spread/3, n_customers)
    longitudes = np.random.normal(city_center[1], lon_spread/3, n_customers)
    
    # Create customer IDs
    customer_ids = [f"CUST_{i:03d}" for i in range(1, n_customers + 1)]
    
    return pd.DataFrame({
        'CustomerID': customer_ids,
        'Latitude': latitudes,
        'Longitude': longitudes
    })

In [6]:
# Create sample data
sample_customers = generate_sample_data(n_customers=150)
print("Sample data created:")
print(sample_customers.head())
print(f"Total customers: {len(sample_customers)}")

Sample data created:
  CustomerID   Latitude  Longitude
0   CUST_001  40.788733 -73.965238
1   CUST_002  40.750596 -73.957630
2   CUST_003  40.797800 -74.039020
3   CUST_004  40.850373 -73.966684
4   CUST_005  40.744837 -73.961862
Total customers: 150


## 1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER

In [7]:
# =============================================================================
# 1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER
# =============================================================================

from clustering.geographic_clustering import GeographicClusteringOptimizer  # Your original code

print("\n" + "="*60)
print("1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER")
print("="*60)

# Initialize with default parameters
optimizer = GeographicClusteringOptimizer()

print("\nDEFAULT PARAMETERS:")
print("- max_customers_per_cluster=20")
print("- max_distance_km=50")
print("- dbscan_eps_km=10")
print("- dbscan_min_samples=3")
print("- use_haversine=True")

# Example 1: Hierarchical clustering
print("\n1a. Hierarchical Clustering (Distance-based):")
hierarchical_result = optimizer.create_hierarchical_clusters(
    sample_customers.copy(),
    method='average',      # OPTIONAL: 'average', 'ward', 'single', 'complete'
    criterion='distance'   # OPTIONAL: 'distance', 'maxclust', 'inconsistent'
)
print(f"Clusters created: {hierarchical_result['cluster'].nunique()}")
print(f"Cluster sizes: {hierarchical_result['cluster'].value_counts().sort_index().head()}")

# Example 2: DBSCAN clustering
print("\n1b. DBSCAN Clustering:")
dbscan_result = optimizer.create_dbscan_clusters(
    sample_customers.copy(),
    return_noise_separately=False  # OPTIONAL: True/False
)
print(f"Clusters created: {dbscan_result['cluster'].nunique()}")

# Example 3: Hybrid clustering
print("\n1c. Hybrid Clustering:")
hybrid_result = optimizer.create_hybrid_clusters(
    sample_customers.copy(),
    dbscan_first=True,                    # OPTIONAL: True/False
    hierarchical_method='average',        # OPTIONAL: linkage method
    hierarchical_criterion='distance'     # OPTIONAL: clustering criterion
)
print(f"Clusters created: {hybrid_result['cluster'].nunique()}")

# Custom parameters example
print("\n1d. Custom Parameters Example:")
custom_optimizer = GeographicClusteringOptimizer(
    max_customers_per_cluster=15,  # REQUIRED
    max_distance_km=30,           # REQUIRED  
    dbscan_eps_km=8,              # OPTIONAL (default=10)
    dbscan_min_samples=2,         # OPTIONAL (default=3)
    use_haversine=False           # OPTIONAL (default=True)
)

custom_result = custom_optimizer.create_hierarchical_clusters(sample_customers.copy())
print(f"Custom clusters created: {custom_result['cluster'].nunique()}")



1. ORIGINAL GEOGRAPHIC CLUSTERING OPTIMIZER

DEFAULT PARAMETERS:
- max_customers_per_cluster=20
- max_distance_km=50
- dbscan_eps_km=10
- dbscan_min_samples=3
- use_haversine=True

1a. Hierarchical Clustering (Distance-based):
Clusters created: 1
Cluster sizes: cluster
1    150
Name: count, dtype: int64

1b. DBSCAN Clustering:
Clusters created: 2

1c. Hybrid Clustering:
Clusters created: 2

1d. Custom Parameters Example:
Custom clusters created: 1


## 2. DIVISIVE HIERARCHICAL CLUSTERING

In [8]:
# =============================================================================
# 2. DIVISIVE HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.divisive_clustering import DivisiveGeographicClustering

print("\n" + "="*60)
print("2. DIVISIVE HIERARCHICAL CLUSTERING")
print("="*60)

divisive_clusterer = DivisiveGeographicClustering(
    max_customers_per_cluster=20,  # REQUIRED
    max_distance_km=50            # REQUIRED
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- max_distance_km=50 (REQUIRED)")

divisive_result = divisive_clusterer.divisive_clustering(sample_customers.copy())
print(f"\nDivisive clusters created: {divisive_result['cluster'].nunique()}")
print(f"Cluster sizes: {divisive_result['cluster'].value_counts().sort_index().head()}")


2. DIVISIVE HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- max_distance_km=50 (REQUIRED)

Divisive clusters created: 12
Cluster sizes: cluster
1    19
2    12
3    18
4     9
5     6
Name: count, dtype: int64


## 3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING

In [9]:
# =============================================================================
# 3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.constrained_hierarchical_clustering import ConstraintBasedClustering

print("\n" + "="*60)
print("3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING")
print("="*60)

constraint_clusterer = ConstraintBasedClustering(
    max_customers_per_cluster=20,  # REQUIRED
    max_distance_km=50,           # REQUIRED
    max_route_distance_km=100     # OPTIONAL (default=100)
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- max_distance_km=50 (REQUIRED)")
print("- max_route_distance_km=100 (OPTIONAL)")

constraint_result = constraint_clusterer.constrained_hierarchical_clustering(sample_customers.copy())
print(f"\nConstraint-based clusters created: {constraint_result['cluster'].nunique()}")
print(f"Cluster sizes: {constraint_result['cluster'].value_counts().sort_index().head()}")


3. CONSTRAINT-BASED HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- max_distance_km=50 (REQUIRED)
- max_route_distance_km=100 (OPTIONAL)

Constraint-based clusters created: 2
Cluster sizes: cluster
-1    130
 8     20
Name: count, dtype: int64


## 4. DENSITY-BASED HIERARCHICAL CLUSTERING

In [10]:
# =============================================================================
# 4. DENSITY-BASED HIERARCHICAL CLUSTERING
# =============================================================================

from clustering.density_hierarchical_clustering import DensityHierarchicalClustering

print("\n" + "="*60)
print("4. DENSITY-BASED HIERARCHICAL CLUSTERING")
print("="*60)

density_clusterer = DensityHierarchicalClustering(
    max_customers_per_cluster=20,  # REQUIRED
    min_cluster_size=3,           # OPTIONAL (default=3)
    k_neighbors=5                 # OPTIONAL (default=5)
)

print("\nPARAMETERS:")
print("- max_customers_per_cluster=20 (REQUIRED)")
print("- min_cluster_size=3 (OPTIONAL)")
print("- k_neighbors=5 (OPTIONAL)")

density_result = density_clusterer.build_density_hierarchy(sample_customers.copy())
print(f"\nDensity-based clusters created: {density_result['cluster'].nunique()}")
print(f"Cluster sizes: {density_result['cluster'].value_counts().sort_index().head()}")



4. DENSITY-BASED HIERARCHICAL CLUSTERING

PARAMETERS:
- max_customers_per_cluster=20 (REQUIRED)
- min_cluster_size=3 (OPTIONAL)
- k_neighbors=5 (OPTIONAL)

Density-based clusters created: 13
Cluster sizes: cluster
-1    18
 1    12
 2    20
 3    20
 4    20
Name: count, dtype: int64


## 5. ROUTE-AWARE CLUSTERING

In [12]:
# =============================================================================
# 5. ROUTE-AWARE CLUSTERING
# =============================================================================
from clustering.route_aware_clustering import RouteAwareClustering

print("\n" + "="*60)
print("5. ROUTE-AWARE CLUSTERING")
print("="*60)

# Define depot location (optional)
depot_location = (40.7589, -73.9851)  # NYC coordinates

route_clusterer = RouteAwareClustering(
    max_customers_per_route=20,      # REQUIRED
    max_route_time_minutes=480,      # OPTIONAL (default=480, 8 hours)
    avg_service_time_minutes=15,     # OPTIONAL (default=15)
    avg_speed_kmh=30,               # OPTIONAL (default=30)
    depot_location=depot_location    # OPTIONAL (default=None)
)

print("\nPARAMETERS:")
print("- max_customers_per_route=20 (REQUIRED)")
print("- max_route_time_minutes=480 (OPTIONAL)")
print("- avg_service_time_minutes=15 (OPTIONAL)")
print("- avg_speed_kmh=30 (OPTIONAL)")
print("- depot_location=(lat, lon) (OPTIONAL)")

# Example 5a: Sweep Algorithm
print("\n5a. Sweep Algorithm Clustering:")
sweep_result = route_clusterer.sweep_algorithm_clustering(sample_customers.copy())
print(f"Sweep clusters created: {sweep_result['cluster'].nunique()}")
print(f"Cluster sizes: {sweep_result['cluster'].value_counts().sort_index().head()}")

# Example 5b: Capacity Constrained Clustering
print("\n5b. Capacity Constrained Clustering:")
capacity_result = route_clusterer.capacity_constrained_clustering(sample_customers.copy())
print(f"Capacity-constrained clusters created: {capacity_result['cluster'].nunique()}")
print(f"Cluster sizes: {capacity_result['cluster'].value_counts().sort_index().head()}")

# Example without depot location
print("\n5c. Route-Aware Clustering without depot:")
no_depot_clusterer = RouteAwareClustering(
    max_customers_per_route=15,
    max_route_time_minutes=360,
    depot_location=None  # Will use geographic center
)
no_depot_result = no_depot_clusterer.capacity_constrained_clustering(sample_customers.copy())
print(f"No-depot clusters created: {no_depot_result['cluster'].nunique()}")



5. ROUTE-AWARE CLUSTERING

PARAMETERS:
- max_customers_per_route=20 (REQUIRED)
- max_route_time_minutes=480 (OPTIONAL)
- avg_service_time_minutes=15 (OPTIONAL)
- avg_speed_kmh=30 (OPTIONAL)
- depot_location=(lat, lon) (OPTIONAL)

5a. Sweep Algorithm Clustering:
Sweep clusters created: 8
Cluster sizes: cluster
1    20
2    20
3    20
4    20
5    20
Name: count, dtype: int64

5b. Capacity Constrained Clustering:
Capacity-constrained clusters created: 8
Cluster sizes: cluster
1    20
2    20
3    20
4    20
5    20
Name: count, dtype: int64

5c. Route-Aware Clustering without depot:
No-depot clusters created: 11


## COMPARISON SUMMARY

In [13]:
# =============================================================================
# COMPARISON SUMMARY
# =============================================================================

print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

results_summary = {
    'Method': [
        'Hierarchical (Distance)',
        'DBSCAN',
        'Hybrid',
        'Divisive',
        'Constraint-Based',
        'Density-Based',
        'Sweep Algorithm',
        'Capacity Constrained'
    ],
    'Clusters': [
        hierarchical_result['cluster'].nunique(),
        dbscan_result['cluster'].nunique(),
        hybrid_result['cluster'].nunique(),
        divisive_result['cluster'].nunique(),
        constraint_result['cluster'].nunique(),
        density_result['cluster'].nunique(),
        sweep_result['cluster'].nunique(),
        capacity_result['cluster'].nunique()
    ]
}

summary_df = pd.DataFrame(results_summary)
print(summary_df)




COMPARISON SUMMARY
                    Method  Clusters
0  Hierarchical (Distance)         1
1                   DBSCAN         2
2                   Hybrid         2
3                 Divisive        12
4         Constraint-Based         2
5            Density-Based        13
6          Sweep Algorithm         8
7     Capacity Constrained         8


## VISUALIZATION

In [16]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

def create_cluster_map(df, lat_col='Latitude', lon_col='Longitude', cluster_col='cluster', 
                     popup_cols=None, tooltip_cols=None, zoom_start=9, 
                     tiles='cartodb positron', radius=5, fill_opacity=0.9, 
                     palette='tableau10'):
    """
    Create a Folium map with clustered markers based on provided DataFrame, with scalable color palettes.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing latitude, longitude, cluster, and optional popup/tooltip columns.
    lat_col : str, optional
        Name of the latitude column (default: 'Latitude').
    lon_col : str, optional
        Name of the longitude column (default: 'Longitude').
    cluster_col : str, optional
        Name of the cluster column (default: 'cluster').
    popup_cols : list of str, optional
        List of column names to include in popups (default: None).
    tooltip_cols : list of str, optional
        List of column names to include in tooltips (default: None).
    zoom_start : int, optional
        Initial zoom level of the map (default: 9).
    tiles : str, optional
        Map tile style (default: 'cartodb positron').
    radius : float, optional
        Marker radius (default: 5).
    fill_opacity : float, optional
        Marker fill opacity (default: 0.9).
    palette : str, optional
        Color palette for clusters. Options: 'tableau10', 'set1', 'retro_metro', 'viridis', 'rainbow' (default: 'tableau10').

    Returns:
    --------
    folium.Map
        A Folium map object with clustered markers.
    """
    # Calculate map center
    lat_m = df[lat_col].mean()
    lon_m = df[lon_col].mean()

    # Initialize map
    cluster_map = folium.Map([lat_m, lon_m], zoom_start=zoom_start, tiles=tiles)

    # Generate color scheme for clusters
    unique_clusters = np.array(sorted(df[cluster_col].unique()))
    n_clusters = len(unique_clusters)

    # Create a mapping from cluster values to color indices
    cluster_to_index = {cluster: idx for idx, cluster in enumerate(unique_clusters)}

    # Use predefined palettes for smaller cluster counts, interpolate for larger ones
    if n_clusters <= 10 and palette in ['tableau10', 'set1', 'retro_metro']:
        if palette == 'tableau10':
            palette_colors = sns.color_palette("tab10", n_colors=min(n_clusters, 10))
            rainbow = [colors.rgb2hex(i) for i in palette_colors]
        elif palette == 'set1':
            palette_colors = sns.color_palette("Set1", n_colors=min(n_clusters, 9))
            rainbow = [colors.rgb2hex(i) for i in palette_colors]
        elif palette == 'retro_metro':
            retro_colors = ['#e60049', '#0bb4ff', '#50e991', '#e6d800', '#9b19f5']
            rainbow = retro_colors[:min(n_clusters, 5)]
        # Extend palette by cycling if needed
        rainbow = (rainbow * (n_clusters // len(rainbow) + 1))[:n_clusters]
    else:
        # Use continuous colormap for large clusters or viridis/rainbow
        if palette == 'viridis':
            palette_colors = cm.viridis(np.linspace(0, 1, n_clusters))
        else:  # Default to rainbow or if explicitly selected
            palette_colors = cm.rainbow(np.linspace(0, 1, n_clusters))
        rainbow = [colors.rgb2hex(i) for i in palette_colors]

    # Add markers to the map
    for _, row in df.iterrows():
        # Prepare popup content
        popup_content = None
        if popup_cols:
            popup_content = '<br>'.join(
                f"{col}: {row[col]}" for col in popup_cols if col in df.columns
            )
            popup_content = folium.Popup(popup_content, parse_html=True)

        # Prepare tooltip content
        tooltip_content = None
        if tooltip_cols:
            tooltip_content = ', '.join(
                f"{row[col]}" for col in tooltip_cols if col in df.columns
            )
            if cluster_col in df.columns:
                tooltip_content += f" - Cluster {row[cluster_col]}"

        # Get color index for the cluster
        cluster_value = row[cluster_col]
        try:
            color_idx = cluster_to_index[cluster_value]
        except KeyError:
            # Fallback to first color if cluster value is invalid
            color_idx = 0

        # Create marker
        folium.vector_layers.CircleMarker(
            [row[lat_col], row[lon_col]],
            radius=radius,
            popup=popup_content,
            tooltip=tooltip_content,
            color=rainbow[color_idx],
            fill=True,
            fill_color=rainbow[color_idx],
            fill_opacity=fill_opacity
        ).add_to(cluster_map)

    return cluster_map

In [37]:
from clustering.plot_cluster import create_cluster_map
 
map_clusters = create_cluster_map(
    divisive_result,
    popup_cols=['CustomerID', 'LGA', 'LCDA'],
    tooltip_cols=['LGA', 'LCDA'],
    palette='tableau10',
    zoom_start=10
).add_child(folium.Marker(location=depot_location, 
                          size = 10, 
                          tooltip='Deport', 
                          icon=folium.Icon(color="green", 
                          icon="home"),))  

map_clusters.save('./maps/nyc_divisive_clustering.html')
map_clusters

## PARAMETER REFERENCE GUIDE

In [14]:
# =============================================================================
# PARAMETER REFERENCE GUIDE
# =============================================================================

print("\n" + "="*60)
print("PARAMETER REFERENCE GUIDE")
print("="*60)

parameter_guide = """
1. GEOGRAPHIC CLUSTERING OPTIMIZER:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: dbscan_eps_km=10, dbscan_min_samples=3, use_haversine=True
   
2. DIVISIVE CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: None
   
3. CONSTRAINT-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: max_route_distance_km=100
   
4. DENSITY-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster
   OPTIONAL: min_cluster_size=3, k_neighbors=5
   
5. ROUTE-AWARE CLUSTERING:
   REQUIRED: max_customers_per_route
   OPTIONAL: max_route_time_minutes=480, avg_service_time_minutes=15,
            avg_speed_kmh=30, depot_location=None

METHOD-SPECIFIC OPTIONAL PARAMETERS:
- create_hierarchical_clusters(): method='average', criterion='distance'
- create_dbscan_clusters(): return_noise_separately=False
- create_hybrid_clusters(): dbscan_first=True, hierarchical_method='average',
                           hierarchical_criterion='distance'
"""

print(parameter_guide)




PARAMETER REFERENCE GUIDE

1. GEOGRAPHIC CLUSTERING OPTIMIZER:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: dbscan_eps_km=10, dbscan_min_samples=3, use_haversine=True
   
2. DIVISIVE CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: None
   
3. CONSTRAINT-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster, max_distance_km
   OPTIONAL: max_route_distance_km=100
   
4. DENSITY-BASED CLUSTERING:
   REQUIRED: max_customers_per_cluster
   OPTIONAL: min_cluster_size=3, k_neighbors=5
   
5. ROUTE-AWARE CLUSTERING:
   REQUIRED: max_customers_per_route
   OPTIONAL: max_route_time_minutes=480, avg_service_time_minutes=15,
            avg_speed_kmh=30, depot_location=None

METHOD-SPECIFIC OPTIONAL PARAMETERS:
- create_hierarchical_clusters(): method='average', criterion='distance'
- create_dbscan_clusters(): return_noise_separately=False
- create_hybrid_clusters(): dbscan_first=True, hierarchical_method='average',
                     

## PRACTICAL USAGE EXAMPLES

In [ ]:
# =============================================================================
# PRACTICAL USAGE EXAMPLES
# =============================================================================

print("\n" + "="*60)
print("PRACTICAL USAGE EXAMPLES")
print("="*60)

print("\nExample 1: Quick start with minimal parameters")
print("=" * 50)
quick_optimizer = GeographicClusteringOptimizer(
    max_customers_per_cluster=15,
    max_distance_km=25
)
quick_result = quick_optimizer.create_hierarchical_clusters(sample_customers.copy())
print(f"Quick clusters: {quick_result['cluster'].nunique()}")

print("\nExample 2: Delivery-focused clustering")
print("=" * 50)
delivery_optimizer = RouteAwareClustering(
    max_customers_per_route=12,
    max_route_time_minutes=300,  # 5 hours
    depot_location=(40.7589, -73.9851)
)
delivery_result = delivery_optimizer.capacity_constrained_clustering(sample_customers.copy())
print(f"Delivery-optimized clusters: {delivery_result['cluster'].nunique()}")

print("\nExample 3: Dense urban area clustering")
print("=" * 50)
urban_optimizer = DensityHierarchicalClustering(
    max_customers_per_cluster=25,
    min_cluster_size=5,
    k_neighbors=8
)
urban_result = urban_optimizer.build_density_hierarchy(sample_customers.copy())
print(f"Urban density clusters: {urban_result['cluster'].nunique()}")

print("\n" + "="*60)
print("EXECUTION COMPLETED SUCCESSFULLY!")
print("="*60)